# SudoQus

En este proyecto se plantea resolver un puzzle tipo Sudoku utilizando el algoritmo cuántico de búsqueda Grover. Para tratar de resolver este reto, hemos optado por dividir el trabajo en tareas más pequeñas y asequibles.

## 1. IMPORTAMOS LIBRERÍAS:

Lo primero que necesitamos es importar las librerías que necesitaremos para trabajar.

In [18]:
from qiskit import *                                    # Imports we need for our project
from qiskit import IBMQ                                 # We will also be simulating in an IBM computer
from qiskit.tools.visualization import plot_histogram
import numpy as np 
import itertools
import operator
%matplotlib inline

## 2. DEFINICIÓN SISTEMA CUÁNTICO:
A continuación creamos el objeto circuito con un número de qubits igual al doble de casillas vacías.

In [12]:
nQbits = 8;                        # Number of qubit of our circuit
q      = QuantumRegister(nQbits)   # List of qubits
c      = ClassicalRegister(nQbits) # List of classical bits 
qc     = QuantumCircuit(q,c)       # Creation of our circuit

Insertamos una puerta hadamard sobre todos los qubits para crear un estado en superposición con todas las posibles soluciones que satisfacen nuestro sistema inicial.

In [ ]:
qc.h(range(nQbits))                # Insert a hadamard gate in each qubit to create the superposition 

Nuestro planteamiento consistirá en incrementar gradualmente la población del estado que corresponda al sudoku resuelto. Para ello se utiliza un algoritmo tipo grover. Grover se fundamenta en dos secciones, un oráculo que invierte el signo del estado correcto y un reflector o modulo de difusión que suaviza la soluciones expureas. Iterando sobre este algoritmo conseguimos que el estado solución destaque frente al resto de estados considerados.

## 3. GROVER + REFLECTOR:

### 3.1 Metodos auxiliares

En el approch que estamos considerando aplicamos el algoritmo grover sobre cada fila, columna y cuadrante, combinando los resultados que generna cada parte.

Las siguientes funciones las utilizamos para generar el conjunto de solciones superpuestas en cada fila/columna. Primero geenramos todas las permutaciones congruentes con los digitos prefijados en cada fila/columna/cuadrante y posteriormente pasamos el resultado a binario para poder exportarlo a qubits.

In [13]:
# Function that creates a list of of the possible permutations given the number(s) we do not want to consider 
def per_gen(classicalB):
    aux      = [0, 1, 2, 3]                             # All the values we consider to be possible
    list_out = []
    for i in aux:
        if i not in classicalB:                         # Ex: if classicalB=[0,1] and i=2 then we will write a 2 in our list_out
            list_out.append(i)
    list_out_2 = list(itertools.permutations(list_out)) # We create the permutations with the filtered numbers
    return list_out_2

#---------------------------------------------------------------------------------------------------------------------#

# Function that converts a list of permutations into its binary values. Ex: [1,2,3] -> ['011011']
def qbit_to_binary(perm):
    perm_bin = []
    for item in list(perm):
        binary_num = ""
        for i in item:
            prm_b = format(i,"b")
            if len(prm_b)==1:                  # If the length of the binary number is 1 we add a 0 to the left to have a constant length of 2
                prm_b = '0' + prm_b
            binary_num = binary_num + prm_b
        perm_bin.append(binary_num)
    return perm_bin

### 3.2 Oraculo y Reflector

Definimos los modulos oraculo y reflectos. Los oraculos se genran combinando oraculos independientes para cada conjunto de estados que queremos detectar en cada fila/columna. 


El modulo de difusión se geenra a partir de la secuencia Hadamard, puerta X, z controll para todas las celda, puerta x y hadamard.

In [ ]:
# Function that creates an adapted oracle to the given list of qubits and list of known numbers in some row/column/block
def oracle(qc, l_qbits, l_perm_bin):
    for item in l_perm_bin:
        cont = 0                                            # Add the necessary X gates at the beginning of the oracle when the value of the char=0
        for j in item:
            if j == '0': 
                qc.x(l_qbits[cont])
            cont+=1
        qc.mcrz(-np.pi,l_qbits[1:len(l_qbits)],l_qbits[0])  # Add a Multiple Controlled Z gate
        cont = 0
        for j in item:                                      # Add the necessary X gates at the end of the oracle when the value of the char=0
            if j == '0':
                qc.x(l_qbits[cont])
            cont+=1
    return qc                                               # Return the given circuit modified

#---------------------------------------------------------------------------------------------------------------------#

# Function that generates a suitable reflector for the list of qubits
def reflector(qc, l_qbits):
    qc.h(l_qbits)                                      # Add hadamard gates in each qubit 
    qc.x(l_qbits)                                      # Add X gates in each qubit
    qc.mcrz(-np.pi,l_qbits[1:len(l_qbits)],l_qbits[0]) # Add a Multiple Controlled Z gate
    qc.x(l_qbits)                                      # Add X gates in each qubit
    qc.h(l_qbits)                                      # Add hadamard gates in each qubit
    return qc                                          # Return the given circuit modified

### 3.3 Algoritmo de Grover

Iteramos sobre los oraculos y reflectores definidos para hayar la solución correcta.

In [24]:
# Function that implements the right ammount of oracles and reflectors to increase the population of the right states
def grover(qc,l_qbits, l_bits):    
    if len(l_qbits) == 2:           # Decides the number of iterations based on the number of qubits
        n_rep = 1
    elif len(l_qbits) == 4:
        n_rep = 1
    else:
        n_rep = 2  
    perm      = per_gen(l_bits)      # Create the permutation for the given bits
    perm_bin  = qbit_to_binary(perm) # Transform the permutation to its binary values
    for i in range(n_rep):          # Repeat oracle and reflector n_rep times
        oracle(qc,l_qbits,perm_bin)
        reflector(qc,[q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7]])
    if len(l_qbits) == 2:           # Final oracle to correct undesired phase
        oracle(qc,l_qbits,perm_bin) 
    return qc                       # Return the given circuit modified


## 4. SOLUCIÓN PARTICULAR:
Definimos nuestro sudoku particular:



In [14]:
#SUDOKU DEFINITION (X : [0,1,2,3]):

#  _______
#  |X1| 0|
#  _______
#  |2 | 1|
#  _______
#  |X2| 3|
#  _____________
#  |1 |X3| 3|X4|
#  _____________

grover(qc, [q[0], q[1]], [0, 1, 2])
grover(qc, [q[0], q[1], q[2], q[3]], [1, 2])
grover(qc, [q[4], q[5], q[6], q[7]], [1, 3])
grover(qc, [q[2], q[3], q[4], q[5]], [1, 3])

['11']
['0011', '1100']
['0010', '1000']
['0010', '1000']


Ejecución de la solución particular que hemos definido:

In [22]:
# Execute in a local backend
qc.measure(q, c)
backend = Aer.get_backend('qasm_simulator')
result  = execute(qc, backend = backend, shots = 2048*128).result()
counts  = result.get_counts()

## 5. INTERPRETACIÓN DE LA SOLUCIÓN

Nos quedamos con las top 5 soluciones. Aquella solcuión con mayor número de cuentas corresponde con la solución del suduku:

In [27]:
sorted_x = sorted(counts.items(), key=operator.itemgetter(1))
print(sorted_x[-5:])

[('11100011', 6184), ('00110011', 6188), ('00000011', 6210), ('01000011', 6769), ('00010011', 10394)]


Descodificamos la solución final de binario a lenguaje sudoku: Nuestra solcuión particular tiene 4 incógnitas, cada una de ellas codificada en 2 qubits. Glosario:

00 - > 0

01 - > 1

10 - > 2

11 - > 3

El resultado se lee en dirección opuesta:
## 00010011 => X4 / X3 / X2 / X1 => 0 / 2 / 0 / 3

In [ ]:
#  _______
#  |*3| 0|
#  _______
#  |2 | 1|
#  _______
#  |*0| 3|
#  _____________
#  |1 |X*2| 3|*0|
#  _____________